# MovieLens 10M Collaborative Genre Tagging
**To do**:
  - try out [this implementation](https://www.onceupondata.com/2019/02/10/nn-collaborative-filtering/) of baseline features. 
  - create object classes for models
  - implement TF 2.0 data classes
  - [paperswithcode link](https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k)
  - [ML 100k state of the art paper](https://arxiv.org/pdf/1706.02263v2.pdf) (RMSE=0.905): details their evaluation method
  
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/James-Leslie/deep-collaborative-filtering/blob/master/tf-movielens10m.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import glob
import os

%matplotlib inline

## Load rating data

In [2]:
path = 'data/ml-10M100K/'  # ML-10M files

all_files = glob.glob(os.path.join(path, "ratings*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [3]:
df.head()

,userId,movieId,rating
0,69587,1005,2.0
1,47904,193,4.0
2,26906,3097,3.5
3,31241,559,1.0
4,69402,2541,4.0


,userId,movieId,rating
0,69587,1005,2.0
1,47904,193,4.0
2,26906,3097,3.5
3,31241,559,1.0
4,69402,2541,4.0


In [4]:
df.shape

(10000054, 3)

In [5]:
print('Number of users:', df.userId.nunique())
print('Number of items:', df.movieId.nunique())
print("Min item rating:", df.rating.min())
print("Max item rating:", df.rating.max())
print("Mean item rating:", df.rating.mean())

Number of users: 69878
Number of items: 10677
Min item rating: 0.5
Max item rating: 5.0
Mean item rating: 3.512421932921562


## Load movie metadata
  - remove 10% as holdout test set

In [6]:
movies = pd.read_csv(path+'movies.tsv', sep='\t')
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,4,Stargate (1994),1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,4,Stargate (1994),1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
movies, movies_holdout = train_test_split(movies, test_size=.1, random_state=42)

---
# Create baseline features
For each user, calculate average user bias - the average difference between the user's rating and the movie's average rating:

$$b_{u} = \dfrac{\sum_{j=1}^{n_u} (r_{uj} - \mu_i)}{n_u}$$

For each item, calculate the difference between its average rating and the average rating of all movies:

$$b_{i} = \dfrac{\sum_{k=1}^{n_i} (r_{ki})}{n_i} - \mu$$

Then, for each interaction, calculate the combined bias:

$$b_{ui} = \dfrac{b_u + b_i}{2}$$

In [9]:
from CGT import get_baseline
?get_baseline

Signature: get_baseline(df, train_index, test_index)
Docstring:
Calculate baseline features from an explicit ratings dataset. Receives a dataframe
and returns train and test splits with added bias column and mean rating value.
User and item biases are calculated as average difference from global mean rating.
Baseline factors are only calculated from training observations, with users or
items that do not appear in train receiving the global average as default.

Args:
    df          : explicit ratings dataframe with columns userId, movieId and rating
    train_index : train index splits taken from KFold.splits()
    test_index  : test index splits taken from KFold.splits()
    
Returns:
    train, test : train/test splits of df, with added bias column
    global_mean : average rating of all training observations
File:      c:\users\jleslie\documents\deep-collaborative-filtering\cgt.py
Type:      function


---
# CGT model
**To do**:
  - Can we avoid re-training rating model on CV fold?
  - Create a grid search function / class

In [10]:
from CGT import compile_genre_model
?compile_genre_model

Signature:
compile_genre_model(
    n_items,
    n_users,
    min_rating,
    max_rating,
    mean_rating,
    n_latent,
    n_hidden_1,
    n_hidden_2,
    activation='relu',
    dropout_1=0.2,
    dropout_2=0.2,
    random_seed=42,
)
Docstring: <no docstring>
File:      c:\users\jleslie\documents\deep-collaborative-filtering\cgt.py
Type:      function


# Classification report

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

### Train model on full dataset, with best hparams

In [12]:
# get baseline predictors for full dataset
train, _, _ = get_baseline(df, df.index, df.index)

# compile both models
model1, model2 = compile_genre_model(
    n_items=df.movieId.nunique(),
    n_users=df.userId.nunique(),
    min_rating=df.rating.min(),
    max_rating=df.rating.max(),
    mean_rating=df.rating.mean(),
    n_latent=200, 
    n_hidden_1=100,
    n_hidden_2=100,
    dropout_1=.15,
    dropout_2=.15
)

# train rating model
ratings = model1.fit(
    x=[train.userId.values, train.movieId.values, train.bias.values],
    y=train.rating.values, 
    batch_size=3072,
    epochs=6,
    verbose=1,
    validation_split=.2
)

# train genre model
genres = model2.fit(
    movies.movieId.values, movies.Drama.values,
    batch_size=256, 
    epochs=5,
    validation_split=.2)

Train on 8000043 samples, validate on 2000011 samples
Epoch 1/6
8000043/8000043 [==============================] - 581s 73us/sample - loss: 0.7211 - val_loss: 0.6896
Epoch 2/6
8000043/8000043 [==============================] - 589s 74us/sample - loss: 0.6653 - val_loss: 0.6604
Epoch 3/6
8000043/8000043 [==============================] - 596s 74us/sample - loss: 0.6263 - val_loss: 0.6452
Epoch 4/6
8000043/8000043 [==============================] - 601s 75us/sample - loss: 0.5925 - val_loss: 0.6374
Epoch 5/6
8000043/8000043 [==============================] - 604s 75us/sample - loss: 0.5635 - val_loss: 0.6354
Epoch 6/6
8000043/8000043 [==============================] - 609s 76us/sample - loss: 0.5393 - val_loss: 0.6365
Train on 7687 samples, validate on 1922 samples
Epoch 1/5
7687/7687 [==============================] - 1s 129us/sample - loss: 0.6306 - accuracy: 0.6520 - AUC: 0.7074 - val_loss: 0.5861 - val_accuracy: 0.6878 - val_AUC: 0.7588
Epoch 2/5
7687/7687 [==========================

### Evaluate on test set

In [23]:
X_test = movies_holdout.movieId.values
y_test = movies_holdout.Drama.values
y_score = pd.DataFrame(model2.predict(X_test))
y_pred = y_score.round().astype('int')

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.70      0.71       551
           1       0.68      0.70      0.69       517

    accuracy                           0.70      1068
   macro avg       0.70      0.70      0.70      1068
weighted avg       0.70      0.70      0.70      1068



In [25]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,384,167
1,154,363


,0,1
0,384,167
1,154,363


In [26]:
movies_holdout['prediction'] = y_pred.values

In [17]:
movies_holdout[['movieId', 'title', 'Drama', 'prediction']].to_csv(path+'holdout_predictions.csv', index=False)

In [18]:
X_train = movies.movieId.values
y_train = movies.Drama.values
train_score = pd.DataFrame(model2.predict(X_train))
train_pred = train_score.round().astype('int')

In [19]:
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.73      0.70      0.71      4790
           1       0.71      0.74      0.72      4819

    accuracy                           0.72      9609
   macro avg       0.72      0.72      0.72      9609
weighted avg       0.72      0.72      0.72      9609



In [20]:
pd.DataFrame(confusion_matrix(y_train, train_pred))

,0,1
0,3354,1436
1,1264,3555


,0,1
0,3354,1436
1,1264,3555


In [21]:
movies['prediction'] = train_pred.values

In [22]:
movies[['movieId', 'title', 'Drama', 'prediction']].to_csv(path+'train_predictions.csv', index=False)

# To DO:
  - ~~extract movie embeddings~~
  - plot PCA of movie embeddings, overlaid with genres

In [82]:
movies = pd.read_csv(path+'movies.tsv', sep='\t').sort_values('movieId')
movies.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,4,Stargate (1994),1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,4,Stargate (1994),1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [83]:
emb = model1.get_layer('embedding')

In [84]:
emb_df = pd.concat(
    (movies, pd.DataFrame(emb.embeddings.numpy(), columns=['e'+str(i) for i in range(200)])),
    axis=1
)

In [85]:
emb_df.head()

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,e190,e191,e192,e193,e194,e195,e196,e197,e198,e199
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,...,0.087806,0.127755,0.020493,0.033983,-0.116912,-0.012477,-0.045383,0.007603,0.150989,-0.148305
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,...,0.077832,0.008988,0.014570,0.073239,0.124648,-0.134136,-0.114936,0.071621,0.278991,0.044786
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,...,-0.046292,-0.049686,0.218707,-0.030238,-0.197417,-0.022804,0.056282,-0.186790,-0.039575,-0.325568
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,...,0.010315,0.058355,-0.088681,0.181235,0.037869,0.079277,0.070823,0.164783,0.237280,0.022873
4,4,Stargate (1994),1,1,0,0,0,0,0,0,...,-0.065126,0.078158,-0.027330,0.022515,0.092744,0.284196,-0.122310,0.033767,-0.013537,0.319222


,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,e190,e191,e192,e193,e194,e195,e196,e197,e198,e199
0,0,Boomerang (1992),0,0,0,0,1,0,0,0,...,0.087806,0.127755,0.020493,0.033983,-0.116912,-0.012477,-0.045383,0.007603,0.150989,-0.148305
1,1,"Net, The (1995)",1,0,0,0,0,1,0,0,...,0.077832,0.008988,0.014570,0.073239,0.124648,-0.134136,-0.114936,0.071621,0.278991,0.044786
2,2,Dumb & Dumber (1994),0,0,0,0,1,0,0,0,...,-0.046292,-0.049686,0.218707,-0.030238,-0.197417,-0.022804,0.056282,-0.186790,-0.039575,-0.325568
3,3,Outbreak (1995),1,0,0,0,0,0,0,1,...,0.010315,0.058355,-0.088681,0.181235,0.037869,0.079277,0.070823,0.164783,0.237280,0.022873
4,4,Stargate (1994),1,1,0,0,0,0,0,0,...,-0.065126,0.078158,-0.027330,0.022515,0.092744,0.284196,-0.122310,0.033767,-0.013537,0.319222


In [86]:
emb_df.to_csv(path+'embeddings.csv', index=False)